# B2C Sales - Data Wrangling and Exploratory Analisis

This project showcases the first step in working with a real-world sales dataset from the B2C retail channel in Argentina.

## 📌 Objective
To clean, transform, and explore sales data using Python, with the goal of building a solid analytical base for further modeling.

---

## 1. Data Wrangling

The dataset that we will be working consists of three different tables:

    AR_PDV.csv | pointsOfSale: Metadata of points of sale (e.g., store type, region, chain)
    AR_PRD.csv | products: Metadata of products (e.g., category, brand, presentation)
    AR_VTA.csv | sales: Weekly sales records with volume and revenue metrics, linked by store and product

The dataset was provided via a number of .zip files uploaded weekly. Our first job is to read and combine all of the .zip files to get three dataframes.

### 1.1. Loading the dataframes

In [1]:
# Import libraries for data wrangling and analysis
import pandas as pd
import zipfile
import os
import numpy as np

In [2]:
# Path one step above, into data folder
path_files = r'../data/B2C Sales - Data Wrangling and EDA'

zip_files = [f for f in os.listdir(path_files) if f.endswith('.zip')]
# Order the list of zip files by their name
zip_files.sort(reverse=False)

print("Zip files: ")
print(zip_files)


Zip files: 
['VENTAS_ANON_20240309.zip', 'VENTAS_ANON_20240316.zip', 'VENTAS_ANON_20240323.zip', 'VENTAS_ANON_20240330.zip', 'VENTAS_ANON_20240406.zip', 'VENTAS_ANON_20240413.zip', 'VENTAS_ANON_20240420.zip', 'VENTAS_ANON_20240427.zip', 'VENTAS_ANON_20240504.zip', 'VENTAS_ANON_20240511.zip', 'VENTAS_ANON_20240518.zip', 'VENTAS_ANON_20240525.zip', 'VENTAS_ANON_20240601.zip', 'VENTAS_ANON_20240608.zip', 'VENTAS_ANON_20240615.zip']


In [3]:
def extract_csv_from_zips(prefix, path_files, zip_files, verbose=True):
    """
    Extract CSV files starting with `prefix` from a list of ZIP files.

    Returns a single DataFrame with all files combined.
    """
    dataframes = []

    for zip_file in zip_files:
        zip_path = os.path.join(path_files, zip_file)
        with zipfile.ZipFile(zip_path, 'r') as z:
            for filename in z.namelist():
                if filename.startswith(prefix) and filename.endswith('.csv'):
#                    if verbose:
#                        print(f"Processing {filename} from {zip_path}")
                    with z.open(filename) as f:
                        df = pd.read_csv(f, delimiter=';')
                        df['file_name'] = filename 
                        dataframes.append(df)

    if not dataframes:
        print(f"No csv files found starting with '{prefix}'.")
        return pd.DataFrame()
    else:
        df_final = pd.concat(dataframes, ignore_index=True)
        if verbose:
            print(f"{prefix} successfully processed. Total rows: {len(df_final)}")
        return df_final

In [4]:
sales = extract_csv_from_zips("AR_VTA", path_files, zip_files)
products = extract_csv_from_zips("AR_PRD", path_files, zip_files)
pointsOfSale = extract_csv_from_zips("AR_PDV", path_files, zip_files)

AR_VTA successfully processed. Total rows: 3615562
AR_PRD successfully processed. Total rows: 32420
AR_PDV successfully processed. Total rows: 27297


In [5]:
sales.head()

,Semana_Inicio_Semana,Codigo_Unico_PDV,PDV_Comparables,Codigo_Barras_SKU,Categoria,Cantidad_Contenido_SKU,Cantidad_de_Venta,Precio_por_Unidade,file_name
0,20230619,ID0001,0,ID0001,ID0001,160.0,2,"1118,40",AR_VTA_20240309.csv
1,20230612,ID0002,1,ID0002,ID0001,320.0,2,"3040,00",AR_VTA_20240309.csv
2,20230522,ID0003,1,ID0003,ID0002,225.0,4,"1760,00",AR_VTA_20240309.csv
3,20230612,ID0004,0,ID0004,ID0003,160.0,2,"1208,18",AR_VTA_20240309.csv
4,20230605,ID0005,0,ID0005,ID0002,225.0,1,"239,25",AR_VTA_20240309.csv


In [6]:
products.head()

,PRD_CODIGO,CATEGORIA_SKU,PROVEEDOR_SKU,MARCA_SKU,NOMBRE_SKU,CODIGO_BARRAS_SKU,PROD_CANT_CONTENIDO,file_name
0,ID0001,ID0001,ID0001,ID0001,ID0001,ID0001,300.0,AR_PRD_20240309.csv
1,ID0002,ID0002,ID0002,ID0002,ID0002,ID0002,500.0,AR_PRD_20240309.csv
2,ID0003,ID0002,ID0002,ID0002,ID0003,ID0003,NaN,AR_PRD_20240309.csv
3,ID0004,ID0003,ID0002,ID0002,ID0004,ID0004,380.0,AR_PRD_20240309.csv
4,ID0005,ID0004,ID0003,ID0003,ID0005,ID0005,375.0,AR_PRD_20240309.csv


In [7]:
pointsOfSale.head()

,CODIGO_PDV,CODIGO_UNICO_PDV,NOMBRE_PDV,DIRECCION_PDV,COMPARABLES_HOY,AREA_P,ZONA_MODELO_B,CLUSTER_PDV,ESTADO_PDV,LOCALIDAD_PDV,ZONA_P,file_name,RUC
0,ID0001,ID0001,ID0001,ID0001,1,Interior,NEA,Autoservicio Mediano,CHACO,RESISTENCIA,CHACO RESISTENCIA,AR_PDV_20240309.csv,NaN
1,ID0002,ID0002,ID0002,ID0002,0,Interior,NOA,NaN,TUCUMAN,LULES,TUCUMAN INTERIOR,AR_PDV_20240309.csv,NaN
2,ID0003,ID0003,ID0003,ID0003,1,Interior,LITORAL,Autoservicio Mediano,ENTRE RIOS,PARANA,ENTRE RIOS PARANA,AR_PDV_20240309.csv,NaN
3,ID0004,ID0004,ID0004,ID0004,0,Metropolitana,CAPITAL FEDERAL,Autoservicio Chico,BUENOS AIRES,CAPITAL FEDERAL,C.A.B.A,AR_PDV_20240309.csv,NaN
4,ID0005,ID0005,ID0005,ID0005,0,Interior,CORDOBA,NaN,CORDOBA,Cordoba,CORDOBA CAPITAL,AR_PDV_20240309.csv,NaN


We now have the three datasets loaded in memory. With this, we move on to the data cleaning.

As a good practice, we leave an internal copy of the raw data as a back-up.

In [8]:
# Save raw data copies for backup
sales_raw = sales.copy()
products_raw = products.copy()
pointsOfSale_raw = pointsOfSale.copy() 

### 1.2 Data Cleaning

Now we move on to the data cleaning, including:
- Drop duplicates
- Check and convert data types
- Normalize key fields
- Assess missing values
- Prepare for joining datasets

---

#### 🧾 Metadata Overview
Below is a quick reference of the columns contained in each of the three dataframes:

##### 📍 pointsOfSale (AR_PDV.csv)
Contains metadata about each point of sale (store), such as its unique identifier, location, and segmentation information.

- *CODIGO_PDV: Internal store code (anonymized)*
- *CODIGO_UNICO_PDV: Unique point-of-sale identifier (anonymized)*
- *NOMBRE_PDV: Store name (anonymized)*
- *DIRECCION_PDV: Store address (anonymized)*
- *COMPARABLES_HOY: Store comparability flag*
- *AREA_P: Geographical area or region*
- *RUC: Tax ID or business ID (optional / not always present)*
- *ZONA_MODELO_B: Internal model segmentation*
- *CLUSTER_PDV: Store cluster (used for grouping stores by behavior or profile)*
- *ESTADO_PDV: Store status (e.g., active, inactive)*
- *LOCALIDAD_PDV: City or locality*
- *ZONA_P: Internal geographic segmentation*
- *file_name: Source file from which the row was extracted*

##### 📦 products (AR_PRD.csv)
Contains product-level metadata, useful for grouping and analysis by brand, category, or SKU.

- *PRD_CODIGO: Internal product code (anonymized)*
- *CATEGORIA_SKU: Product category (anonymized)*
- *PROVEEDOR_SKU: Product supplier or manufacturer (anonymized)*
- *MARCA_SKU: Brand name (anonymized)*
- *NOMBRE_SKU: Product description or label (anonymized)*
- *CODIGO_BARRAS_SKU: Barcode identifier (anonymized)*
- *PROD_CANT_CONTENIDO: Net content or pack size (e.g., grams, liters)*
- *file_name: Source file from which the row was extracted*

##### 🛒 sales (AR_VTA.csv)
Contains transactional sales data, with weekly granularity and links to stores and products.

- *Semana_Inicio_Semana: Starting date of the week (used for time series analysis)*
- *Codigo_Unico_PDV: Unique store identifier (anonymized)*
- *PDV_Comparables: Flag indicating whether the store is part of comparable set*
- *Codigo_Barras_SKU: Product identifier (anonymized)*
- *Categoria: Product category (anonymized)*
- *Cantidad_Contenido_SKU: Net content of the SKU*
- *Cantidad_de_Venta: Units sold*
- *Precio_por_Unidade: Price per unit sold*
- *file_name: Source file from which the row was extracted*

---

#### 1.2.1 Drop duplicates
From the metadata, we can identify the key fields from each table:
- pointsOfSale: Either CODIGO_PDV or CODIGO_UNICO_PDV → We choose CODIGO_UNICO_PDV to join to the sales dataframe
- products: Either PRD_CODIGO or CODIGO_BARRAS_SKU → We choose CODIGO_BARRAS_SKU to join to the sales dataframe
- sales: Semana_Inicio_Semana + Codigo_Unico_PDV + Codigo_Barras_SKU

In [9]:
# Identify key fields for each table
pointsOfSale_key = ['CODIGO_UNICO_PDV']
products_key = ['CODIGO_BARRAS_SKU']
sales_key = ['Semana_Inicio_Semana', 'Codigo_Unico_PDV', 'Codigo_Barras_SKU']

In [10]:
# Check for duplicates in pointsOfSale
print("Duplicates in pointOfSale:",
      pointsOfSale.duplicated(subset=pointsOfSale_key).sum(),
      "out of", len(pointsOfSale),
      "(", round(pointsOfSale.duplicated(subset=pointsOfSale_key).sum() / len(pointsOfSale) * 100, 1), "% )")
print("Duplicates in products:",
      products.duplicated(subset=products_key).sum(),
      "out of", len(products),
      "(", round(products.duplicated(subset=products_key).sum() / len(products) * 100, 1), "% )")
print("Duplicates in sales:",
      sales.duplicated(subset=sales_key).sum(),
      "out of", len(sales),
      "(", round(sales.duplicated(subset=sales_key).sum() / len(sales) * 100, 1), "% )")

Duplicates in pointOfSale: 25463 out of 27297 ( 93.3 % )
Duplicates in products: 30264 out of 32420 ( 93.3 % )
Duplicates in sales: 860447 out of 3615562 ( 23.8 % )


In both pointofSale and products, the duplicates are expected, because we merged metadata tables.
However in sales, duplicates are not expected, which means that in each individual file there was data for more than one week.


Here we make an important assumption that should always be validated: We will eliminate the duplicates by always leaving the last occurance, by file name (which is correspondent to when the file was created). However, this is only correct in context for this data. 

In [11]:
# Eliminate duplicates by keeping the last occurrence
pointsOfSale = pointsOfSale.drop_duplicates(subset=pointsOfSale_key, keep='last')
products = products.drop_duplicates(subset=products_key, keep='last')
sales = sales.drop_duplicates(subset=sales_key, keep='last')

In [12]:
# Check for duplicates in pointsOfSale
print("Duplicates in pointOfSale:",
      pointsOfSale.duplicated(subset=pointsOfSale_key).sum(),
      "out of", len(pointsOfSale),
      "(", round(pointsOfSale.duplicated(subset=pointsOfSale_key).sum() / len(pointsOfSale) * 100, 1), "% )")
print("Duplicates in products:",
      products.duplicated(subset=products_key).sum(),
      "out of", len(products),
      "(", round(products.duplicated(subset=products_key).sum() / len(products) * 100, 1), "% )")
print("Duplicates in sales:",
      sales.duplicated(subset=sales_key).sum(),
      "out of", len(sales),
      "(", round(sales.duplicated(subset=sales_key).sum() / len(sales) * 100, 1), "% )")

Duplicates in pointOfSale: 0 out of 1834 ( 0.0 % )
Duplicates in products: 0 out of 2156 ( 0.0 % )
Duplicates in sales: 0 out of 2755115 ( 0.0 % )


Now rows have no duplicates, however we still have either duplicated fields, or multiple ID fields in our tables. We will also remove the filename column from the tables.

In [13]:
# Eliminate duplicated fields
cols_to_drop = {
    "pointsOfSale": [
        'CODIGO_PDV',
        'file_name'],
    "products": [
        'PRD_CODIGO',
        'file_name'],
    "sales": [
              'PDV_Comparables',
              'Categoria',
              'Cantidad_Contenido_SKU',
              'file_name']
}
for df_name, cols in cols_to_drop.items():
    if df_name == "pointsOfSale":
        pointsOfSale.drop(columns=cols, inplace=True)
    elif df_name == "products":
        products.drop(columns=cols, inplace=True)
    elif df_name == "sales":
        sales.drop(columns=cols, inplace=True)

#### 1.2.2 Check and convert Data Types

In [14]:
# Check data types
print("Data types in pointsOfSale:")
print(pointsOfSale.dtypes) 
print("\nData types in products:")
print(products.dtypes)
print("\nData types in sales:")
print(sales.dtypes)

Data types in pointsOfSale:
CODIGO_UNICO_PDV    object
NOMBRE_PDV          object
DIRECCION_PDV       object
COMPARABLES_HOY      int64
AREA_P              object
ZONA_MODELO_B       object
CLUSTER_PDV         object
ESTADO_PDV          object
LOCALIDAD_PDV       object
ZONA_P              object
RUC                 object
dtype: object

Data types in products:
CATEGORIA_SKU           object
PROVEEDOR_SKU           object
MARCA_SKU               object
NOMBRE_SKU              object
CODIGO_BARRAS_SKU       object
PROD_CANT_CONTENIDO    float64
dtype: object

Data types in sales:
Semana_Inicio_Semana     int64
Codigo_Unico_PDV        object
Codigo_Barras_SKU       object
Cantidad_de_Venta        int64
Precio_por_Unidade      object
dtype: object


Most of the data types are already appropriate for analysis, but we will convert some columns to string and float as needed.

The column COMPARABLES_HOY will be converted to boolean.

Lastly to note, the column Precio_por_Unidade in sales is currently a string, but it should be a float. We will convert it to float after removing the currency symbol and commas.


In [ ]:
# Convert columns to string in all dataframes
cols_to_str = {
    "pointsOfSale": ['CODIGO_UNICO_PDV', 'NOMBRE_PDV', 'DIRECCION_PDV',
       'AREA_P', 'ZONA_MODELO_B', 'CLUSTER_PDV', 'ESTADO_PDV', 'LOCALIDAD_PDV',
       'ZONA_P', 'RUC'],
    "products": ['CATEGORIA_SKU', 'PROVEEDOR_SKU', 'MARCA_SKU', 'NOMBRE_SKU',
       'CODIGO_BARRAS_SKU'],
    "sales": ['Codigo_Unico_PDV', 'Codigo_Barras_SKU']
}

for col in cols_to_str["pointsOfSale"]:
    pointsOfSale[col] = pointsOfSale[col].astype(str)

for col in cols_to_str["products"]:
    products[col] = products[col].astype(str)

for col in cols_to_str["sales"]:
    sales[col] = sales[col].astype(str)

In [37]:
# Convert COMPARABLES_HOY to boolean
pointsOfSale['COMPARABLES_HOY'] = pointsOfSale['COMPARABLES_HOY'].replace({'SI': True, 'NO': False})
pointsOfSale['COMPARABLES_HOY'] = pointsOfSale['COMPARABLES_HOY'].astype(bool)

In [38]:
# Replace commas with dots in numeric columns and convert to float
cols_to_replace = {
    "pointsOfSale": [],
    "products": ['PROD_CANT_CONTENIDO'],
    "sales": ['Cantidad_de_Venta', 'Precio_por_Unidade']
}
for col in cols_to_replace["pointsOfSale"]:
    pointsOfSale[col] = pointsOfSale[col].astype(str).str.replace(',', '.').astype(float)
for col in cols_to_replace["products"]:
    products[col] = products[col].astype(str).str.replace(',', '.').astype(float)
for col in cols_to_replace["sales"]:
    sales[col] = sales[col].astype(str).str.replace(',', '.').astype(float)

In [42]:
#Convert Semana_Inicio_Semana to datetime
sales['Semana_Inicio_Semana'] = pd.to_datetime(sales['Semana_Inicio_Semana'], format='%Y%m%d')

In [45]:
# Check data types
print("Data types in pointsOfSale:")
print(pointsOfSale.dtypes) 
print("\nData types in products:")
print(products.dtypes)
print("\nData types in sales:")
print(sales.dtypes)

Data types in pointsOfSale:
CODIGO_UNICO_PDV    object
NOMBRE_PDV          object
DIRECCION_PDV       object
COMPARABLES_HOY       bool
AREA_P              object
ZONA_MODELO_B       object
CLUSTER_PDV         object
ESTADO_PDV          object
LOCALIDAD_PDV       object
ZONA_P              object
RUC                 object
dtype: object

Data types in products:
CATEGORIA_SKU           object
PROVEEDOR_SKU           object
MARCA_SKU               object
NOMBRE_SKU              object
CODIGO_BARRAS_SKU       object
PROD_CANT_CONTENIDO    float64
dtype: object

Data types in sales:
Semana_Inicio_Semana    datetime64[ns]
Codigo_Unico_PDV                object
Codigo_Barras_SKU               object
Cantidad_de_Venta              float64
Precio_por_Unidade             float64
dtype: object


#### 1.2.3 Normalize key fields (coming soon)

#### 1.2.4 Assess missing values (coming soon)

#### 1.2.5 Prepare for joining datasets (coming soon)

### 1.3 Preparing for joins (coming soon)

After cleaning each individual dataset, we verify that the key fields are suitable for merging:

- AR_VTA can be joined with AR_PDV using `Codigo_Unico_PDV`
- AR_VTA can be joined with AR_PRD using `Codigo_Barras_SKU`

## 2. Exploratory Data Analysis (coming soon)